In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Amazon

### T5

In [2]:
# 라이브러리 임포트
! pip install optuna catboost

import pandas as pd
import numpy as np
import catboost as cb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/PADA model code/PADA/data/amazon/amazon.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/PADA model code/PADA/embedding/amazon_T5.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: CatBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'Logloss',
        'eval_metric': 'PRAUC', # PR AUC를 직접 모니터링
        'bootstrap_type': 'Bernoulli',
        'verbose': 0,
        'random_seed': Config.RANDOM_STATE,
        'iterations': trial.suggest_int('iterations', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'depth': trial.suggest_int('depth', 3, 12),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True), # (lambda_l2)
        #'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: CatBoostClassifier 사용 및 GPU 설정 (task_type='GPU')
    model = cb.CatBoostClassifier(task_type='GPU', **params)

    # 🌟 변경점: CatBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              early_stopping_rounds=100,
              verbose=False)

    # CatBoost는 PRAUC를 직접 계산하므로, 최적화 단계의 점수를 바로 반환
    score = model.get_best_score()['validation']['PRAUC']

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    baseline_model = cb.CatBoostClassifier(task_type='GPU', random_seed=config.RANDOM_STATE, verbose=0)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    final_model = cb.CatBoostClassifier(task_type='GPU', objective='Logloss', verbose=0, random_seed=config.RANDOM_STATE,bootstrap_type='Bernoulli', **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join('/content/drive/MyDrive/PADA model code/PADA/results/s2/amazon/CatBoost', "T5_amazon_with_s2_predictions_CatBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}' 파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 26.6 MB/s eta 0:00:00
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 71941건, 테스트용: 17986건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 06:23:40,163] A new study created in memory with name: no-name-ab99ee40-6cbd-4887-aa9a-55853b93a150


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 06:23:49,199] Trial 0 finished with value: 0.2912206657625446 and parameters: {'iterations': 1000, 'learning_rate': 0.022442501202564132, 'depth': 8, 'l2_leaf_reg': 0.0019053723302337449, 'subsample': 0.7098100210499023}. Best is trial 0 with value: 0.2912206657625446.
Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 06:24:07,166] Trial 1 finished with value: 0.29557598074315 and parameters: {'iterations': 1200, 'learning_rate': 0.0040430030361591, 'depth': 7, 'l2_leaf_reg': 4.528790317223022e-06, 'subsample': 0.8708910716954632}. Best is trial 1 with value: 0.29557598074315.
Default metric period is 5 because PRAUC is/are 


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
          iterations: 1400
       learning_rate: 0.016145435584898706
               depth: 9
         l2_leaf_reg: 0.7522018243340284
           subsample: 0.9061760560385682

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.3295   0.7799    0.1069
Optuna Tuned  0.3261   0.7820    0.0907

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/PADA model code/PADA/results/s2/amazon/CatBoost/T5_amazon_with_s2_predictions_CatBoost.csv' 파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!


### DistilBERT

In [2]:
# 라이브러리 임포트
! pip install optuna catboost

import pandas as pd
import numpy as np
import catboost as cb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/PADA model code/PADA/data/amazon/amazon.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/PADA model code/PADA/embedding/amazon_DistilBERT.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: CatBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'Logloss',
        'eval_metric': 'PRAUC', # PR AUC를 직접 모니터링
        'bootstrap_type': 'Bernoulli',
        'verbose': 0,
        'random_seed': Config.RANDOM_STATE,
        'iterations': trial.suggest_int('iterations', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'depth': trial.suggest_int('depth', 3, 12),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True), # (lambda_l2)
        #'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: CatBoostClassifier 사용 및 GPU 설정 (task_type='GPU')
    model = cb.CatBoostClassifier(task_type='GPU', **params)

    # 🌟 변경점: CatBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              early_stopping_rounds=100,
              verbose=False)

    # CatBoost는 PRAUC를 직접 계산하므로, 최적화 단계의 점수를 바로 반환
    score = model.get_best_score()['validation']['PRAUC']

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    baseline_model = cb.CatBoostClassifier(task_type='GPU', random_seed=config.RANDOM_STATE, verbose=0)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    final_model = cb.CatBoostClassifier(task_type='GPU', objective='Logloss', verbose=0, random_seed=config.RANDOM_STATE,bootstrap_type='Bernoulli', **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join(os.path.dirname(config.CSV_FILE_PATH), "DistilBERT_amazon_with_s2_predictions_CatBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}' 파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 71941건, 테스트용: 17986건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 07:15:11,414] A new study created in memory with name: no-name-e2c00cff-7114-42ce-b5ed-f87bfe4444c5


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 07:15:25,208] Trial 0 finished with value: 0.27087633605371486 and parameters: {'iterations': 400, 'learning_rate': 0.034956922707076565, 'depth': 9, 'l2_leaf_reg': 0.00015721801540305994, 'subsample': 0.7422761780294713}. Best is trial 0 with value: 0.27087633605371486.
Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 07:16:30,097] Trial 1 finished with value: 0.28340520807599967 and parameters: {'iterations': 1800, 'learning_rate': 0.0017367401721670992, 'depth': 9, 'l2_leaf_reg': 0.0003380581123859918, 'subsample': 0.9345401660469452}. Best is trial 1 with value: 0.28340520807599967.
Default metric period is 5 because PR


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
          iterations: 1300
       learning_rate: 0.017084170832190732
               depth: 4
         l2_leaf_reg: 1.6167192772584735
           subsample: 0.6040294239239935

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.3334   0.7821    0.1312
Optuna Tuned  0.3274   0.7800    0.1355

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/PADA model code/PADA/data/amazon/DistilBERT_amazon_with_s2_predictions_CatBoost.csv' 파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!


### RoBERTa

In [3]:
import pandas as pd
import numpy as np
import catboost as cb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/PADA model code/PADA/data/amazon/amazon.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/PADA model code/PADA/embedding/amazon_RoBERTa.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: CatBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'Logloss',
        'eval_metric': 'PRAUC', # PR AUC를 직접 모니터링
        'bootstrap_type': 'Bernoulli',
        'verbose': 0,
        'random_seed': Config.RANDOM_STATE,
        'iterations': trial.suggest_int('iterations', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'depth': trial.suggest_int('depth', 3, 12),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True), # (lambda_l2)
        #'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: CatBoostClassifier 사용 및 GPU 설정 (task_type='GPU')
    model = cb.CatBoostClassifier(task_type='GPU', **params)

    # 🌟 변경점: CatBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              early_stopping_rounds=100,
              verbose=False)

    # CatBoost는 PRAUC를 직접 계산하므로, 최적화 단계의 점수를 바로 반환
    score = model.get_best_score()['validation']['PRAUC']

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    baseline_model = cb.CatBoostClassifier(task_type='GPU', random_seed=config.RANDOM_STATE, verbose=0)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    final_model = cb.CatBoostClassifier(task_type='GPU', objective='Logloss', verbose=0, random_seed=config.RANDOM_STATE,bootstrap_type='Bernoulli', **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join('/content/drive/MyDrive/PADA model code/PADA/results/s2/amazon/CatBoost', "RoBERTa_amazon_with_s2_predictions_CatBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}' 파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 71941건, 테스트용: 17986건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 07:42:56,522] A new study created in memory with name: no-name-e70d5b9b-9f31-4b70-b2b8-e84765aa4dd8


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 07:43:11,614] Trial 0 finished with value: 0.2904650412063041 and parameters: {'iterations': 700, 'learning_rate': 0.041064070087408006, 'depth': 4, 'l2_leaf_reg': 1.1904954560039274e-08, 'subsample': 0.6732495239721827}. Best is trial 0 with value: 0.2904650412063041.
Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 07:43:47,824] Trial 1 finished with value: 0.19980672639353955 and parameters: {'iterations': 1500, 'learning_rate': 0.19537334837404136, 'depth': 12, 'l2_leaf_reg': 1.66215667299954e-07, 'subsample': 0.8912076695591317}. Best is trial 0 with value: 0.2904650412063041.
Default metric period is 5 because PRAUC i


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
          iterations: 700
       learning_rate: 0.041064070087408006
               depth: 4
         l2_leaf_reg: 1.1904954560039274e-08
           subsample: 0.6732495239721827

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.3295   0.7751    0.1443
Optuna Tuned  0.3254   0.7722    0.1483

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/PADA model code/PADA/results/s2/amazon/CatBoost/RoBERTa_amazon_with_s2_predictions_CatBoost.csv' 파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!


### SentenceBERT

In [4]:
import pandas as pd
import numpy as np
import catboost as cb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/PADA model code/PADA/data/amazon/amazon.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/PADA model code/PADA/embedding/amazon_SentenceBERT.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: CatBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'Logloss',
        'eval_metric': 'PRAUC', # PR AUC를 직접 모니터링
        'bootstrap_type': 'Bernoulli',
        'verbose': 0,
        'random_seed': Config.RANDOM_STATE,
        'iterations': trial.suggest_int('iterations', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'depth': trial.suggest_int('depth', 3, 12),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True), # (lambda_l2)
        #'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: CatBoostClassifier 사용 및 GPU 설정 (task_type='GPU')
    model = cb.CatBoostClassifier(task_type='GPU', **params)

    # 🌟 변경점: CatBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              early_stopping_rounds=100,
              verbose=False)

    # CatBoost는 PRAUC를 직접 계산하므로, 최적화 단계의 점수를 바로 반환
    score = model.get_best_score()['validation']['PRAUC']

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    baseline_model = cb.CatBoostClassifier(task_type='GPU', random_seed=config.RANDOM_STATE, verbose=0)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    final_model = cb.CatBoostClassifier(task_type='GPU', objective='Logloss', verbose=0, random_seed=config.RANDOM_STATE,bootstrap_type='Bernoulli', **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join('/content/drive/MyDrive/PADA model code/PADA/results/s2/amazon/CatBoost', "SentenceBERT_amazon_with_s2_predictions_CatBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}' 파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 71941건, 테스트용: 17986건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 08:04:15,657] A new study created in memory with name: no-name-d8037d5e-39d0-417b-a3eb-53dc260d7618


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 08:05:03,675] Trial 0 finished with value: 0.24094471371657702 and parameters: {'iterations': 1500, 'learning_rate': 0.0775744259870355, 'depth': 12, 'l2_leaf_reg': 0.24756923507321377, 'subsample': 0.7844204574243723}. Best is trial 0 with value: 0.24094471371657702.
Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 08:05:34,845] Trial 1 finished with value: 0.2748484709363147 and parameters: {'iterations': 600, 'learning_rate': 0.0875011287994535, 'depth': 5, 'l2_leaf_reg': 2.5813007492734093e-08, 'subsample': 0.6361427521102891}. Best is trial 1 with value: 0.2748484709363147.
Default metric period is 5 because PRAUC is/a


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
          iterations: 1000
       learning_rate: 0.09403549308396385
               depth: 3
         l2_leaf_reg: 0.039586874788199125
           subsample: 0.7292667634809752

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.3180   0.7735    0.0929
Optuna Tuned  0.3093   0.7713    0.1206

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/PADA model code/PADA/results/s2/amazon/CatBoost/SentenceBERT_amazon_with_s2_predictions_CatBoost.csv' 파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!


### BERT

In [6]:
import pandas as pd
import numpy as np
import catboost as cb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/PADA model code/PADA/data/amazon/amazon.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/PADA model code/PADA/embedding/amazon_BERT.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: CatBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'Logloss',
        'eval_metric': 'PRAUC', # PR AUC를 직접 모니터링
        'bootstrap_type': 'Bernoulli',
        'verbose': 0,
        'random_seed': Config.RANDOM_STATE,
        'iterations': trial.suggest_int('iterations', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'depth': trial.suggest_int('depth', 3, 12),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True), # (lambda_l2)
        #'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: CatBoostClassifier 사용 및 GPU 설정 (task_type='GPU')
    model = cb.CatBoostClassifier(task_type='GPU', **params)

    # 🌟 변경점: CatBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              early_stopping_rounds=100,
              verbose=False)

    # CatBoost는 PRAUC를 직접 계산하므로, 최적화 단계의 점수를 바로 반환
    score = model.get_best_score()['validation']['PRAUC']

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    baseline_model = cb.CatBoostClassifier(task_type='GPU', random_seed=config.RANDOM_STATE, verbose=0)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    final_model = cb.CatBoostClassifier(task_type='GPU', objective='Logloss', verbose=0, random_seed=config.RANDOM_STATE,bootstrap_type='Bernoulli', **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join(os.path.dirname('/content/drive/MyDrive/PADA model code/PADA/results/s2/amazon/CatBoost'), "BERT_amazon_with_s2_predictions_CatBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}' 파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 71941건, 테스트용: 17986건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 08:36:30,359] A new study created in memory with name: no-name-cd0259f8-0d73-4251-aef2-b5671f655400


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 08:38:55,600] Trial 0 finished with value: 0.26916149128014344 and parameters: {'iterations': 1300, 'learning_rate': 0.010529971759443995, 'depth': 12, 'l2_leaf_reg': 0.0002310846933488445, 'subsample': 0.9271424978526581}. Best is trial 0 with value: 0.26916149128014344.
Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 08:40:39,173] Trial 1 finished with value: 0.266296191822134 and parameters: {'iterations': 700, 'learning_rate': 0.0014865764761603964, 'depth': 12, 'l2_leaf_reg': 1.2493469732270045e-07, 'subsample': 0.6692761972941429}. Best is trial 0 with value: 0.26916149128014344.
Default metric period is 5 because PR


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
          iterations: 900
       learning_rate: 0.0374044955954863
               depth: 4
         l2_leaf_reg: 2.2289484245245495e-05
           subsample: 0.743851992167047

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.3300   0.7794    0.1331
Optuna Tuned  0.3276   0.7770    0.1398

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장


TypeError: dirname() takes 1 positional argument but 2 were given

# Coursera

### T5

In [1]:
# 라이브러리 임포트
! pip install optuna catboost

import pandas as pd
import numpy as np
import catboost as cb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/PADA model code/PADA/data/coursera/coursera.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/PADA model code/PADA/embedding/coursera_T5.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: CatBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'Logloss',
        'eval_metric': 'PRAUC', # PR AUC를 직접 모니터링
        'bootstrap_type': 'Bernoulli',
        'verbose': 0,
        'random_seed': Config.RANDOM_STATE,
        'iterations': trial.suggest_int('iterations', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'depth': trial.suggest_int('depth', 3, 12),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True), # (lambda_l2)
        #'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: CatBoostClassifier 사용 및 GPU 설정 (task_type='GPU')
    model = cb.CatBoostClassifier(task_type='GPU', **params)

    # 🌟 변경점: CatBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              early_stopping_rounds=100,
              verbose=False)

    # CatBoost는 PRAUC를 직접 계산하므로, 최적화 단계의 점수를 바로 반환
    score = model.get_best_score()['validation']['PRAUC']

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    baseline_model = cb.CatBoostClassifier(task_type='GPU', random_seed=config.RANDOM_STATE, verbose=0)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    final_model = cb.CatBoostClassifier(task_type='GPU', objective='Logloss', verbose=0, random_seed=config.RANDOM_STATE,bootstrap_type='Bernoulli', **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join(os.path.dirname(config.CSV_FILE_PATH), "T5_coursera_with_s2_predictions_CatBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}' 파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 26.6 MB/s eta 0:00:00
Mounted at /content/drive
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 97108건, 테스트용: 24278건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 10:49:26,546] A new study created in memory with name: no-name-c719f61b-e1ec-40f0-b4de-82be954fa1fd


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 10:49:33,284] Trial 0 finished with value: 0.20497754524879958 and parameters: {'iterations': 1900, 'learning_rate': 0.03009129869418295, 'depth': 3, 'l2_leaf_reg': 2.2587702726742427e-05, 'subsample': 0.6255519729943188}. Best is trial 0 with value: 0.20497754524879958.
Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 10:50:02,837] Trial 1 finished with value: 0.18546060298039146 and parameters: {'iterations': 1400, 'learning_rate': 0.001069385032155691, 'depth': 11, 'l2_leaf_reg': 0.00034651321794132645, 'subsample': 0.8848302263231096}. Best is trial 0 with value: 0.20497754524879958.
Default metric period is 5 because P


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
          iterations: 1600
       learning_rate: 0.0213388836155453
               depth: 10
         l2_leaf_reg: 3.7410131330941443
           subsample: 0.6386898656503163

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.2268   0.7925    0.0888
Optuna Tuned  0.2256   0.7977    0.0513

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/PADA model code/PADA/data/coursera/T5_coursera_with_s2_predictions_CatBoost.csv' 파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!


### DistilBERT

In [2]:
# 라이브러리 임포트
! pip install optuna catboost

import pandas as pd
import numpy as np
import catboost as cb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/PADA model code/PADA/data/coursera/coursera.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/PADA model code/PADA/embedding/coursera_DistilBERT.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: CatBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'Logloss',
        'eval_metric': 'PRAUC', # PR AUC를 직접 모니터링
        'bootstrap_type': 'Bernoulli',
        'verbose': 0,
        'random_seed': Config.RANDOM_STATE,
        'iterations': trial.suggest_int('iterations', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'depth': trial.suggest_int('depth', 3, 12),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True), # (lambda_l2)
        #'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: CatBoostClassifier 사용 및 GPU 설정 (task_type='GPU')
    model = cb.CatBoostClassifier(task_type='GPU', **params)

    # 🌟 변경점: CatBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              early_stopping_rounds=100,
              verbose=False)

    # CatBoost는 PRAUC를 직접 계산하므로, 최적화 단계의 점수를 바로 반환
    score = model.get_best_score()['validation']['PRAUC']

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    baseline_model = cb.CatBoostClassifier(task_type='GPU', random_seed=config.RANDOM_STATE, verbose=0)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    final_model = cb.CatBoostClassifier(task_type='GPU', objective='Logloss', verbose=0, random_seed=config.RANDOM_STATE,bootstrap_type='Bernoulli', **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join('/content/drive/MyDrive/PADA model code/PADA/results/s2/coursera/CatBoost', "DistilBERT_coursera_with_s2_predictions_CatBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}' 파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 97108건, 테스트용: 24278건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 11:17:49,279] A new study created in memory with name: no-name-20b9e553-8ece-4ce9-8a7b-0e008a13154a


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 11:18:19,928] Trial 0 finished with value: 0.19918304035073547 and parameters: {'iterations': 2000, 'learning_rate': 0.0046203985649756205, 'depth': 7, 'l2_leaf_reg': 5.6884496280204235, 'subsample': 0.9379805863103097}. Best is trial 0 with value: 0.19918304035073547.
Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 11:18:43,739] Trial 1 finished with value: 0.16056373517389208 and parameters: {'iterations': 800, 'learning_rate': 0.029755833146263955, 'depth': 11, 'l2_leaf_reg': 1.1518519790545944e-06, 'subsample': 0.8812560353245527}. Best is trial 0 with value: 0.19918304035073547.
Default metric period is 5 because PRAU


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
          iterations: 1800
       learning_rate: 0.009411849844121096
               depth: 11
         l2_leaf_reg: 7.87305916751052
           subsample: 0.9985680851669291

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.2067   0.7816    0.0732
Optuna Tuned  0.2104   0.7872    0.0496

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/PADA model code/PADA/results/s2/coursera/CatBoost/DistilBERT_coursera_with_s2_predictions_CatBoost.csv' 파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!


### RoBERTa

In [4]:
import pandas as pd
import numpy as np
import catboost as cb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/PADA model code/PADA/data/coursera/coursera.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/PADA model code/PADA/embedding/coursera_RoBERTa.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: CatBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'Logloss',
        'eval_metric': 'PRAUC', # PR AUC를 직접 모니터링
        'bootstrap_type': 'Bernoulli',
        'verbose': 0,
        'random_seed': Config.RANDOM_STATE,
        'iterations': trial.suggest_int('iterations', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'depth': trial.suggest_int('depth', 3, 12),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True), # (lambda_l2)
        #'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: CatBoostClassifier 사용 및 GPU 설정 (task_type='GPU')
    model = cb.CatBoostClassifier(task_type='GPU', **params)

    # 🌟 변경점: CatBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              early_stopping_rounds=100,
              verbose=False)

    # CatBoost는 PRAUC를 직접 계산하므로, 최적화 단계의 점수를 바로 반환
    score = model.get_best_score()['validation']['PRAUC']

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    baseline_model = cb.CatBoostClassifier(task_type='GPU', random_seed=config.RANDOM_STATE, verbose=0)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    final_model = cb.CatBoostClassifier(task_type='GPU', objective='Logloss', verbose=0, random_seed=config.RANDOM_STATE,bootstrap_type='Bernoulli', **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join('/content/drive/MyDrive/PADA model code/PADA/results/s2/coursera/CatBoost', "RoBERTa_coursera_with_s2_predictions_CatBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}' 파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 97108건, 테스트용: 24278건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 12:00:25,404] A new study created in memory with name: no-name-2f4f3a10-b156-4089-8809-aa96aaf6b33e


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 12:00:32,872] Trial 0 finished with value: 0.135821145774275 and parameters: {'iterations': 200, 'learning_rate': 0.257484120589303, 'depth': 10, 'l2_leaf_reg': 3.669354859263083e-05, 'subsample': 0.7732495541945539}. Best is trial 0 with value: 0.135821145774275.
Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 12:01:04,804] Trial 1 finished with value: 0.17734150806581517 and parameters: {'iterations': 1900, 'learning_rate': 0.012206930198840131, 'depth': 10, 'l2_leaf_reg': 0.0005084349624082418, 'subsample': 0.7913429253883288}. Best is trial 1 with value: 0.17734150806581517.
Default metric period is 5 because PRAUC is/


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
          iterations: 1900
       learning_rate: 0.015955580872806654
               depth: 7
         l2_leaf_reg: 0.3341043224984155
           subsample: 0.8419827885847413

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.2056   0.7691    0.0709
Optuna Tuned  0.1996   0.7695    0.0662

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/PADA model code/PADA/results/s2/coursera/CatBoost/RoBERTa_coursera_with_s2_predictions_CatBoost.csv' 파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!


### SentenceBERT

In [2]:
! pip install optuna catboost
import pandas as pd
import numpy as np
import catboost as cb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/PADA model code/PADA/data/coursera/coursera.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/PADA model code/PADA/embedding/coursera_SentenceBERT.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: CatBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'Logloss',
        'eval_metric': 'PRAUC', # PR AUC를 직접 모니터링
        'bootstrap_type': 'Bernoulli',
        'verbose': 0,
        'random_seed': Config.RANDOM_STATE,
        'iterations': trial.suggest_int('iterations', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'depth': trial.suggest_int('depth', 3, 12),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True), # (lambda_l2)
        #'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: CatBoostClassifier 사용 및 GPU 설정 (task_type='GPU')
    model = cb.CatBoostClassifier(task_type='GPU', **params)

    # 🌟 변경점: CatBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              early_stopping_rounds=100,
              verbose=False)

    # CatBoost는 PRAUC를 직접 계산하므로, 최적화 단계의 점수를 바로 반환
    score = model.get_best_score()['validation']['PRAUC']

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    baseline_model = cb.CatBoostClassifier(task_type='GPU', random_seed=config.RANDOM_STATE, verbose=0)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    final_model = cb.CatBoostClassifier(task_type='GPU', objective='Logloss', verbose=0, random_seed=config.RANDOM_STATE,bootstrap_type='Bernoulli', **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join('/content/drive/MyDrive/PADA model code/PADA/results/s2/coursera/CatBoost', "SentenceBERT_coursera_with_s2_predictions_CatBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}' 파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 27.5 MB/s eta 0:00:00
Mounted at /content/drive
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 97108건, 테스트용: 24278건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 12:44:22,812] A new study created in memory with name: no-name-88adecf8-6da8-4cdf-b685-86539dedf963


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 12:44:58,520] Trial 0 finished with value: 0.18569181163130724 and parameters: {'iterations': 1000, 'learning_rate': 0.00951661762613915, 'depth': 4, 'l2_leaf_reg': 0.6671635728907307, 'subsample': 0.6625173714425118}. Best is trial 0 with value: 0.18569181163130724.
Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 12:45:33,559] Trial 1 finished with value: 0.1848065561390344 and parameters: {'iterations': 1400, 'learning_rate': 0.026409319351995347, 'depth': 9, 'l2_leaf_reg': 1.5434511547726174e-08, 'subsample': 0.7496768266439516}. Best is trial 0 with value: 0.18569181163130724.
Default metric period is 5 because PRAUC i


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
          iterations: 1400
       learning_rate: 0.028722575877608424
               depth: 11
         l2_leaf_reg: 8.580751706394562
           subsample: 0.6849697541204055

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.2063   0.7823    0.0530
Optuna Tuned  0.2104   0.7856    0.0146

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/PADA model code/PADA/results/s2/coursera/CatBoost/SentenceBERT_coursera_with_s2_predictions_CatBoost.csv' 파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!


### BERT

In [3]:
import pandas as pd
import numpy as np
import catboost as cb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/PADA model code/PADA/data/coursera/coursera.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/PADA model code/PADA/embedding/coursera_BERT.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: CatBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'Logloss',
        'eval_metric': 'PRAUC', # PR AUC를 직접 모니터링
        'bootstrap_type': 'Bernoulli',
        'verbose': 0,
        'random_seed': Config.RANDOM_STATE,
        'iterations': trial.suggest_int('iterations', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'depth': trial.suggest_int('depth', 3, 12),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True), # (lambda_l2)
        #'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: CatBoostClassifier 사용 및 GPU 설정 (task_type='GPU')
    model = cb.CatBoostClassifier(task_type='GPU', **params)

    # 🌟 변경점: CatBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              early_stopping_rounds=100,
              verbose=False)

    # CatBoost는 PRAUC를 직접 계산하므로, 최적화 단계의 점수를 바로 반환
    score = model.get_best_score()['validation']['PRAUC']

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    baseline_model = cb.CatBoostClassifier(task_type='GPU', random_seed=config.RANDOM_STATE, verbose=0)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    final_model = cb.CatBoostClassifier(task_type='GPU', objective='Logloss', verbose=0, random_seed=config.RANDOM_STATE,bootstrap_type='Bernoulli', **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join('/content/drive/MyDrive/PADA model code/PADA/results/s2/coursera/CatBoost', "BERT_coursera_with_s2_predictions_CatBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}' 파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 97108건, 테스트용: 24278건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 13:24:43,269] A new study created in memory with name: no-name-5c1eb34c-8ca8-4962-8341-4a3af91e3360


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 13:24:53,857] Trial 0 finished with value: 0.1784821650624031 and parameters: {'iterations': 900, 'learning_rate': 0.001264239383044784, 'depth': 4, 'l2_leaf_reg': 0.00046524848663185366, 'subsample': 0.7030387087371074}. Best is trial 0 with value: 0.1784821650624031.
Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 13:25:00,525] Trial 1 finished with value: 0.18812602929790204 and parameters: {'iterations': 300, 'learning_rate': 0.04776124855798635, 'depth': 7, 'l2_leaf_reg': 0.02287271682122708, 'subsample': 0.6032766890500155}. Best is trial 1 with value: 0.18812602929790204.
Default metric period is 5 because PRAUC is/


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
          iterations: 1400
       learning_rate: 0.011234903064857896
               depth: 8
         l2_leaf_reg: 2.405032409470223
           subsample: 0.8764639454538223

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.2058   0.7807    0.0678
Optuna Tuned  0.2123   0.7858    0.0666

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/PADA model code/PADA/results/s2/coursera/CatBoost/BERT_coursera_with_s2_predictions_CatBoost.csv' 파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!


# Audible

### T5

In [4]:
# 라이브러리 임포트
! pip install optuna catboost

import pandas as pd
import numpy as np
import catboost as cb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/PADA model code/PADA/data/audible/audible.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/PADA model code/PADA/embedding/audible_T5.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: CatBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'Logloss',
        'eval_metric': 'PRAUC', # PR AUC를 직접 모니터링
        'bootstrap_type': 'Bernoulli',
        'verbose': 0,
        'random_seed': Config.RANDOM_STATE,
        'iterations': trial.suggest_int('iterations', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'depth': trial.suggest_int('depth', 3, 12),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True), # (lambda_l2)
        #'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: CatBoostClassifier 사용 및 GPU 설정 (task_type='GPU')
    model = cb.CatBoostClassifier(task_type='GPU', **params)

    # 🌟 변경점: CatBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              early_stopping_rounds=100,
              verbose=False)

    # CatBoost는 PRAUC를 직접 계산하므로, 최적화 단계의 점수를 바로 반환
    score = model.get_best_score()['validation']['PRAUC']

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    baseline_model = cb.CatBoostClassifier(task_type='GPU', random_seed=config.RANDOM_STATE, verbose=0)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    final_model = cb.CatBoostClassifier(task_type='GPU', objective='Logloss', verbose=0, random_seed=config.RANDOM_STATE,bootstrap_type='Bernoulli', **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join('/content/drive/MyDrive/PADA model code/PADA/results/s2/audible/s2/CatBoost', "T5_audible_with_s2_predictions_CatBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}' 파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 74391건, 테스트용: 18598건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 13:48:55,486] A new study created in memory with name: no-name-37e7a7b1-52fa-460f-8c9d-3f3a20ed563a


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 13:48:59,118] Trial 0 finished with value: 0.15649982709253654 and parameters: {'iterations': 200, 'learning_rate': 0.07084554480603182, 'depth': 4, 'l2_leaf_reg': 1.1256233566871785, 'subsample': 0.9968755738519844}. Best is trial 0 with value: 0.15649982709253654.
Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 13:49:04,994] Trial 1 finished with value: 0.13424596061969743 and parameters: {'iterations': 500, 'learning_rate': 0.2681249916493653, 'depth': 9, 'l2_leaf_reg': 3.851857184783566e-08, 'subsample': 0.6008236272639909}. Best is trial 0 with value: 0.15649982709253654.
Default metric period is 5 because PRAUC is/ar


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
          iterations: 800
       learning_rate: 0.003644399363597806
               depth: 3
         l2_leaf_reg: 0.6898472891975261
           subsample: 0.9209998758223894

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1694   0.6828    0.0029
Optuna Tuned  0.1519   0.6559    0.0000

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장


OSError: Cannot save file into a non-existent directory: '/content/drive/MyDrive/PADA model code/PADA/results/s2/audible/s2/CatBoost'

### DistilBERT

In [7]:
# 라이브러리 임포트
! pip install optuna catboost

import pandas as pd
import numpy as np
import catboost as cb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/PADA model code/PADA/data/audible/audible.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/PADA model code/PADA/embedding/audible_DistilBERT.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: CatBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'Logloss',
        'eval_metric': 'PRAUC', # PR AUC를 직접 모니터링
        'bootstrap_type': 'Bernoulli',
        'verbose': 0,
        'random_seed': Config.RANDOM_STATE,
        'iterations': trial.suggest_int('iterations', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'depth': trial.suggest_int('depth', 3, 12),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True), # (lambda_l2)
        #'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: CatBoostClassifier 사용 및 GPU 설정 (task_type='GPU')
    model = cb.CatBoostClassifier(task_type='GPU', **params)

    # 🌟 변경점: CatBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              early_stopping_rounds=100,
              verbose=False)

    # CatBoost는 PRAUC를 직접 계산하므로, 최적화 단계의 점수를 바로 반환
    score = model.get_best_score()['validation']['PRAUC']

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    baseline_model = cb.CatBoostClassifier(task_type='GPU', random_seed=config.RANDOM_STATE, verbose=0)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    final_model = cb.CatBoostClassifier(task_type='GPU', objective='Logloss', verbose=0, random_seed=config.RANDOM_STATE,bootstrap_type='Bernoulli', **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join('/content/drive/MyDrive/PADA model code/PADA/results/s2/audible/CatBoost', "DistilBERT_audible_with_s2_predictions_CatBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}' 파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 74391건, 테스트용: 18598건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 17:56:14,112] A new study created in memory with name: no-name-9d99f9a4-d8c4-4712-b0e2-78794257d129


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 17:56:17,315] Trial 0 finished with value: 0.14288583621977058 and parameters: {'iterations': 1200, 'learning_rate': 0.11792949214117437, 'depth': 3, 'l2_leaf_reg': 1.2231042897490199, 'subsample': 0.9371465093084315}. Best is trial 0 with value: 0.14288583621977058.
Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 17:56:27,364] Trial 1 finished with value: 0.10465859244903027 and parameters: {'iterations': 1400, 'learning_rate': 0.07292715079767148, 'depth': 11, 'l2_leaf_reg': 7.495075996049427e-06, 'subsample': 0.6105061848595601}. Best is trial 0 with value: 0.14288583621977058.
Default metric period is 5 because PRAUC i


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
          iterations: 500
       learning_rate: 0.035550018293643085
               depth: 4
         l2_leaf_reg: 9.500555311629904
           subsample: 0.9027971218077325

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1537   0.6629    0.0015
Optuna Tuned  0.1525   0.6592    0.0000

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/PADA model code/PADA/results/s2/audible/CatBoost/DistilBERT_audible_with_s2_predictions_CatBoost.csv' 파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!


### RoBERTa

In [2]:
import pandas as pd
import numpy as np
import catboost as cb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/PADA model code/PADA/data/audible/audible.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/PADA model code/PADA/embedding/audible_RoBERTa.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: CatBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'Logloss',
        'eval_metric': 'PRAUC', # PR AUC를 직접 모니터링
        'bootstrap_type': 'Bernoulli',
        'verbose': 0,
        'random_seed': Config.RANDOM_STATE,
        'iterations': trial.suggest_int('iterations', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'depth': trial.suggest_int('depth', 3, 12),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True), # (lambda_l2)
        #'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: CatBoostClassifier 사용 및 GPU 설정 (task_type='GPU')
    model = cb.CatBoostClassifier(task_type='GPU', **params)

    # 🌟 변경점: CatBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              early_stopping_rounds=100,
              verbose=False)

    # CatBoost는 PRAUC를 직접 계산하므로, 최적화 단계의 점수를 바로 반환
    score = model.get_best_score()['validation']['PRAUC']

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    baseline_model = cb.CatBoostClassifier(task_type='GPU', random_seed=config.RANDOM_STATE, verbose=0)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    final_model = cb.CatBoostClassifier(task_type='GPU', objective='Logloss', verbose=0, random_seed=config.RANDOM_STATE,bootstrap_type='Bernoulli', **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join('/content/drive/MyDrive/PADA model code/PADA/results/s2/audible/CatBoost', "RoBERTa_audible_with_s2_predictions_CatBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}' 파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 74391건, 테스트용: 18598건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 15:42:53,628] A new study created in memory with name: no-name-3949be48-626f-459e-a1ad-ed1c06a8c20d


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 15:42:57,445] Trial 0 finished with value: 0.1335225390226436 and parameters: {'iterations': 200, 'learning_rate': 0.009055803386677268, 'depth': 5, 'l2_leaf_reg': 0.03876120544189095, 'subsample': 0.7586878588131031}. Best is trial 0 with value: 0.1335225390226436.
Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 15:43:08,570] Trial 1 finished with value: 0.14422590767000623 and parameters: {'iterations': 1900, 'learning_rate': 0.011840236514609878, 'depth': 5, 'l2_leaf_reg': 0.00026665778228334884, 'subsample': 0.637453645536763}. Best is trial 1 with value: 0.14422590767000623.
Default metric period is 5 because PRAUC is


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
          iterations: 1100
       learning_rate: 0.07906913627654913
               depth: 3
         l2_leaf_reg: 5.566314362704277e-05
           subsample: 0.6925313664479653

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1555   0.6649    0.0044
Optuna Tuned  0.1519   0.6617    0.0044

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/PADA model code/PADA/results/s2/audible/CatBoost/RoBERTa_audible_with_s2_predictions_CatBoost.csv' 파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!


### SentenceBERT

In [3]:
import pandas as pd
import numpy as np
import catboost as cb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/PADA model code/PADA/data/audible/audible.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/PADA model code/PADA/embedding/audible_SentenceBERT.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: CatBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'Logloss',
        'eval_metric': 'PRAUC', # PR AUC를 직접 모니터링
        'bootstrap_type': 'Bernoulli',
        'verbose': 0,
        'random_seed': Config.RANDOM_STATE,
        'iterations': trial.suggest_int('iterations', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'depth': trial.suggest_int('depth', 3, 12),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True), # (lambda_l2)
        #'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: CatBoostClassifier 사용 및 GPU 설정 (task_type='GPU')
    model = cb.CatBoostClassifier(task_type='GPU', **params)

    # 🌟 변경점: CatBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              early_stopping_rounds=100,
              verbose=False)

    # CatBoost는 PRAUC를 직접 계산하므로, 최적화 단계의 점수를 바로 반환
    score = model.get_best_score()['validation']['PRAUC']

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    baseline_model = cb.CatBoostClassifier(task_type='GPU', random_seed=config.RANDOM_STATE, verbose=0)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    final_model = cb.CatBoostClassifier(task_type='GPU', objective='Logloss', verbose=0, random_seed=config.RANDOM_STATE,bootstrap_type='Bernoulli', **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join('/content/drive/MyDrive/PADA model code/PADA/results/s2/audible/CatBoost', "SentenceBERT_audible_with_s2_predictions_CatBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}' 파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 74391건, 테스트용: 18598건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 15:52:54,732] A new study created in memory with name: no-name-087f3246-32ec-440d-81db-238851ae6b33


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 15:53:18,426] Trial 0 finished with value: 0.13030414233231705 and parameters: {'iterations': 1800, 'learning_rate': 0.09437037453838931, 'depth': 7, 'l2_leaf_reg': 0.06817754777427268, 'subsample': 0.6760083324208365}. Best is trial 0 with value: 0.13030414233231705.
Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 15:53:58,843] Trial 1 finished with value: 0.12111925342797662 and parameters: {'iterations': 1800, 'learning_rate': 0.02235588553925731, 'depth': 12, 'l2_leaf_reg': 0.31424555827254386, 'subsample': 0.8233952237556498}. Best is trial 0 with value: 0.13030414233231705.
Default metric period is 5 because PRAUC is


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
          iterations: 1900
       learning_rate: 0.015624330593461716
               depth: 7
         l2_leaf_reg: 2.467834211007821
           subsample: 0.9300902599344077

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1566   0.6684    0.0015
Optuna Tuned  0.1586   0.6713    0.0059

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/PADA model code/PADA/results/s2/audible/CatBoost/SentenceBERT_audible_with_s2_predictions_CatBoost.csv' 파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!


### BERT

In [4]:
import pandas as pd
import numpy as np
import catboost as cb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/PADA model code/PADA/data/audible/audible.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/PADA model code/PADA/embedding/audible_BERT.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: CatBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'Logloss',
        'eval_metric': 'PRAUC', # PR AUC를 직접 모니터링
        'bootstrap_type': 'Bernoulli',
        'verbose': 0,
        'random_seed': Config.RANDOM_STATE,
        'iterations': trial.suggest_int('iterations', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'depth': trial.suggest_int('depth', 3, 12),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True), # (lambda_l2)
        #'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: CatBoostClassifier 사용 및 GPU 설정 (task_type='GPU')
    model = cb.CatBoostClassifier(task_type='GPU', **params)

    # 🌟 변경점: CatBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              early_stopping_rounds=100,
              verbose=False)

    # CatBoost는 PRAUC를 직접 계산하므로, 최적화 단계의 점수를 바로 반환
    score = model.get_best_score()['validation']['PRAUC']

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    baseline_model = cb.CatBoostClassifier(task_type='GPU', random_seed=config.RANDOM_STATE, verbose=0)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    final_model = cb.CatBoostClassifier(task_type='GPU', objective='Logloss', verbose=0, random_seed=config.RANDOM_STATE,bootstrap_type='Bernoulli', **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join('/content/drive/MyDrive/PADA model code/PADA/results/s2/audible/CatBoost', "BERT_audible_with_s2_predictions_CatBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}' 파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 74391건, 테스트용: 18598건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 16:19:31,296] A new study created in memory with name: no-name-ce9ecf53-7024-4da0-8951-b45aa57ee18a


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 16:19:34,203] Trial 0 finished with value: 0.147599345348165 and parameters: {'iterations': 700, 'learning_rate': 0.007570901703559071, 'depth': 4, 'l2_leaf_reg': 5.1658812196235345e-08, 'subsample': 0.7399036870433893}. Best is trial 0 with value: 0.147599345348165.
Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 16:19:46,669] Trial 1 finished with value: 0.13295484020481085 and parameters: {'iterations': 300, 'learning_rate': 0.0015609686723010137, 'depth': 10, 'l2_leaf_reg': 0.0830134292937953, 'subsample': 0.7132608560351891}. Best is trial 0 with value: 0.147599345348165.
Default metric period is 5 because PRAUC is/ar


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
          iterations: 800
       learning_rate: 0.02178121467282486
               depth: 5
         l2_leaf_reg: 0.011241886682174278
           subsample: 0.8236285707203299

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1543   0.6650    0.0000
Optuna Tuned  0.1527   0.6619    0.0015

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/PADA model code/PADA/results/s2/audible/CatBoost/BERT_audible_with_s2_predictions_CatBoost.csv' 파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!


# Hotel

### T5

In [5]:
# 라이브러리 임포트
! pip install optuna catboost

import pandas as pd
import numpy as np
import catboost as cb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/PADA model code/PADA/data/hotel/hotel.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/PADA model code/PADA/embedding/hotel_T5.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: CatBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'Logloss',
        'eval_metric': 'PRAUC', # PR AUC를 직접 모니터링
        'bootstrap_type': 'Bernoulli',
        'verbose': 0,
        'random_seed': Config.RANDOM_STATE,
        'iterations': trial.suggest_int('iterations', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'depth': trial.suggest_int('depth', 3, 12),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True), # (lambda_l2)
        #'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: CatBoostClassifier 사용 및 GPU 설정 (task_type='GPU')
    model = cb.CatBoostClassifier(task_type='GPU', **params)

    # 🌟 변경점: CatBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              early_stopping_rounds=100,
              verbose=False)

    # CatBoost는 PRAUC를 직접 계산하므로, 최적화 단계의 점수를 바로 반환
    score = model.get_best_score()['validation']['PRAUC']

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    baseline_model = cb.CatBoostClassifier(task_type='GPU', random_seed=config.RANDOM_STATE, verbose=0)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    final_model = cb.CatBoostClassifier(task_type='GPU', objective='Logloss', verbose=0, random_seed=config.RANDOM_STATE,bootstrap_type='Bernoulli', **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join('/content/drive/MyDrive/PADA model code/PADA/results/s2/hotel/CatBoost', "T5_hotel_with_s2_predictions_CatBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}' 파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 71604건, 테스트용: 17901건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 16:26:19,859] A new study created in memory with name: no-name-04382276-004d-4966-8997-11a68187f91e


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 16:26:22,644] Trial 0 finished with value: 0.15442193358370768 and parameters: {'iterations': 1500, 'learning_rate': 0.02720430427954744, 'depth': 4, 'l2_leaf_reg': 7.563319719613042e-07, 'subsample': 0.8721132578764464}. Best is trial 0 with value: 0.15442193358370768.
Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 16:26:26,158] Trial 1 finished with value: 0.1492002836157929 and parameters: {'iterations': 100, 'learning_rate': 0.008901996598711354, 'depth': 7, 'l2_leaf_reg': 1.7136657491543084, 'subsample': 0.9698100411786689}. Best is trial 0 with value: 0.15442193358370768.
Default metric period is 5 because PRAUC is/


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
          iterations: 700
       learning_rate: 0.2952246439089331
               depth: 3
         l2_leaf_reg: 0.017996159280865374
           subsample: 0.908665455147199

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1628   0.6322    0.0000
Optuna Tuned  0.1281   0.5999    0.0222

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장


OSError: Cannot save file into a non-existent directory: '/content/drive/MyDrive/PADA model code/PADA/results/s2/hotel/CatBoost'

### DistilBERT

In [3]:
# 라이브러리 임포트
! pip install optuna catboost

import pandas as pd
import numpy as np
import catboost as cb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/PADA model code/PADA/data/hotel/hotel.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/PADA model code/PADA/embedding/hotel_DistilBERT.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: CatBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'Logloss',
        'eval_metric': 'PRAUC', # PR AUC를 직접 모니터링
        'bootstrap_type': 'Bernoulli',
        'verbose': 0,
        'random_seed': Config.RANDOM_STATE,
        'iterations': trial.suggest_int('iterations', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'depth': trial.suggest_int('depth', 3, 12),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True), # (lambda_l2)
        #'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: CatBoostClassifier 사용 및 GPU 설정 (task_type='GPU')
    model = cb.CatBoostClassifier(task_type='GPU', **params)

    # 🌟 변경점: CatBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              early_stopping_rounds=100,
              verbose=False)

    # CatBoost는 PRAUC를 직접 계산하므로, 최적화 단계의 점수를 바로 반환
    score = model.get_best_score()['validation']['PRAUC']

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    baseline_model = cb.CatBoostClassifier(task_type='GPU', random_seed=config.RANDOM_STATE, verbose=0)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    final_model = cb.CatBoostClassifier(task_type='GPU', objective='Logloss', verbose=0, random_seed=config.RANDOM_STATE,bootstrap_type='Bernoulli', **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join('/content/drive/MyDrive/PADA model code/PADA/results/s2/hotel/CatBoost', "DistilBERT_hotel_with_s2_predictions_CatBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}' 파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 25.1 MB/s eta 0:00:00
Mounted at /content/drive
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 71604건, 테스트용: 17901건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 16:57:23,475] A new study created in memory with name: no-name-15e739a0-b45a-4a8a-9daa-7e6d1b76d9a8


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 16:57:26,477] Trial 0 finished with value: 0.15066611159694396 and parameters: {'iterations': 500, 'learning_rate': 0.17801475649179624, 'depth': 5, 'l2_leaf_reg': 0.5479022497026351, 'subsample': 0.9960061238527956}. Best is trial 0 with value: 0.15066611159694396.
Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 16:57:38,511] Trial 1 finished with value: 0.15310524358334537 and parameters: {'iterations': 1800, 'learning_rate': 0.0017514124633542427, 'depth': 8, 'l2_leaf_reg': 1.3026415917740594e-08, 'subsample': 0.617786369716268}. Best is trial 1 with value: 0.15310524358334537.
Default metric period is 5 because PRAUC i


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
          iterations: 600
       learning_rate: 0.0026706821611464557
               depth: 3
         l2_leaf_reg: 1.6222955177092791e-07
           subsample: 0.7317020359644316

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1588   0.6287    0.0012
Optuna Tuned  0.1565   0.6230    0.0000

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/PADA model code/PADA/results/s2/hotel/CatBoost/DistilBERT_hotel_with_s2_predictions_CatBoost.csv' 파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!


### RoBERTa

In [4]:
import pandas as pd
import numpy as np
import catboost as cb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/PADA model code/PADA/data/hotel/hotel.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/PADA model code/PADA/embedding/hotel_RoBERTa.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: CatBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'Logloss',
        'eval_metric': 'PRAUC', # PR AUC를 직접 모니터링
        'bootstrap_type': 'Bernoulli',
        'verbose': 0,
        'random_seed': Config.RANDOM_STATE,
        'iterations': trial.suggest_int('iterations', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'depth': trial.suggest_int('depth', 3, 12),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True), # (lambda_l2)
        #'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: CatBoostClassifier 사용 및 GPU 설정 (task_type='GPU')
    model = cb.CatBoostClassifier(task_type='GPU', **params)

    # 🌟 변경점: CatBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              early_stopping_rounds=100,
              verbose=False)

    # CatBoost는 PRAUC를 직접 계산하므로, 최적화 단계의 점수를 바로 반환
    score = model.get_best_score()['validation']['PRAUC']

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    baseline_model = cb.CatBoostClassifier(task_type='GPU', random_seed=config.RANDOM_STATE, verbose=0)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    final_model = cb.CatBoostClassifier(task_type='GPU', objective='Logloss', verbose=0, random_seed=config.RANDOM_STATE,bootstrap_type='Bernoulli', **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join('/content/drive/MyDrive/PADA model code/PADA/results/s2/hotel/CatBoost', "RoBERTa_hotel_with_s2_predictions_CatBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}' 파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 71604건, 테스트용: 17901건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 17:01:36,564] A new study created in memory with name: no-name-606309ad-309c-438a-a92b-1b8681f1b5c3


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 17:01:41,172] Trial 0 finished with value: 0.14523144042163355 and parameters: {'iterations': 1600, 'learning_rate': 0.01917867938376795, 'depth': 8, 'l2_leaf_reg': 2.6944267793082313e-06, 'subsample': 0.6370867526131683}. Best is trial 0 with value: 0.14523144042163355.
Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 17:01:48,138] Trial 1 finished with value: 0.1324338803132318 and parameters: {'iterations': 900, 'learning_rate': 0.039669939834444265, 'depth': 10, 'l2_leaf_reg': 1.3322056200915874e-08, 'subsample': 0.7901172734793185}. Best is trial 0 with value: 0.14523144042163355.
Default metric period is 5 because PRA


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
          iterations: 100
       learning_rate: 0.26380055777123307
               depth: 4
         l2_leaf_reg: 6.405434035925874
           subsample: 0.8834342481633488

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1559   0.6224     0.000
Optuna Tuned  0.1489   0.6162     0.006

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/PADA model code/PADA/results/s2/hotel/CatBoost/RoBERTa_hotel_with_s2_predictions_CatBoost.csv' 파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!


### SentenceBERT

In [5]:
import pandas as pd
import numpy as np
import catboost as cb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/PADA model code/PADA/data/hotel/hotel.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/PADA model code/PADA/embedding/hotel_SentenceBERT.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: CatBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'Logloss',
        'eval_metric': 'PRAUC', # PR AUC를 직접 모니터링
        'bootstrap_type': 'Bernoulli',
        'verbose': 0,
        'random_seed': Config.RANDOM_STATE,
        'iterations': trial.suggest_int('iterations', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'depth': trial.suggest_int('depth', 3, 12),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True), # (lambda_l2)
        #'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: CatBoostClassifier 사용 및 GPU 설정 (task_type='GPU')
    model = cb.CatBoostClassifier(task_type='GPU', **params)

    # 🌟 변경점: CatBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              early_stopping_rounds=100,
              verbose=False)

    # CatBoost는 PRAUC를 직접 계산하므로, 최적화 단계의 점수를 바로 반환
    score = model.get_best_score()['validation']['PRAUC']

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    baseline_model = cb.CatBoostClassifier(task_type='GPU', random_seed=config.RANDOM_STATE, verbose=0)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    final_model = cb.CatBoostClassifier(task_type='GPU', objective='Logloss', verbose=0, random_seed=config.RANDOM_STATE,bootstrap_type='Bernoulli', **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join('/content/drive/MyDrive/PADA model code/PADA/results/s2/hotel/CatBoost', "SentenceBERT_hotel_with_s2_predictions_CatBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}' 파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 71604건, 테스트용: 17901건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 17:12:29,259] A new study created in memory with name: no-name-842168a5-9d27-4308-8f21-a13de5439174


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 17:12:52,469] Trial 0 finished with value: 0.14739738609598665 and parameters: {'iterations': 1800, 'learning_rate': 0.005626512661559711, 'depth': 4, 'l2_leaf_reg': 1.6196110629140679, 'subsample': 0.695261479870711}. Best is trial 0 with value: 0.14739738609598665.
Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 17:13:13,970] Trial 1 finished with value: 0.14370233484939868 and parameters: {'iterations': 100, 'learning_rate': 0.0023998044714954488, 'depth': 5, 'l2_leaf_reg': 6.787170900107042e-05, 'subsample': 0.7665906120600654}. Best is trial 0 with value: 0.14739738609598665.
Default metric period is 5 because PRAUC i


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
          iterations: 400
       learning_rate: 0.04860213404715092
               depth: 6
         l2_leaf_reg: 0.23041940233600475
           subsample: 0.7271368554998889

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1536   0.6260    0.0000
Optuna Tuned  0.1512   0.6171    0.0012

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/PADA model code/PADA/results/s2/hotel/CatBoost/SentenceBERT_hotel_with_s2_predictions_CatBoost.csv' 파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!


### BERT

In [6]:
import pandas as pd
import numpy as np
import catboost as cb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report
import os
import warnings
from tqdm.auto import tqdm
import time

# --- 기본 설정 ---
warnings.filterwarnings('ignore')
# optuna.logging.set_verbosity(optuna.logging.WARNING)

class Config:
    """실행에 필요한 설정값을 관리합니다."""
    # 🌟 TODO: 자신의 파일 전체 경로를 아래에 직접 입력해주세요.
    CSV_FILE_PATH = '/content/drive/MyDrive/PADA model code/PADA/data/hotel/hotel.csv'
    # 🌟 모든 토큰 정보가 담긴 3차원 Raw 임베딩 파일 경로
    EMBEDDING_PATH = "/content/drive/MyDrive/PADA model code/PADA/embedding/hotel_BERT.npy"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'

# --- 데이터 로드 준비 ---
from google.colab import drive
drive.mount('/content/drive')

config = Config()

# --- Step 1: 데이터 로드 및 분할 ---
print("Step 1: 데이터 로드 및 분할 중...")
try:
    df = pd.read_csv(config.CSV_FILE_PATH)
    labels = df[config.TARGET_COLUMN].values
    embeddings = np.load(config.EMBEDDING_PATH)
    assert len(df) == len(embeddings)
except Exception as e:
    print(f"🔥 파일 로드 실패: {e}"); exit()

indices = np.arange(len(df))
train_indices, test_indices = train_test_split(
    indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels
)
X_train, X_test = embeddings[train_indices], embeddings[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]
print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")


def objective(trial, X, y):
    """Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복 호출하는 함수"""
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y
    )

    # 🌟 변경점: CatBoost 하이퍼파라미터 탐색 공간으로 수정
    params = {
        'objective': 'Logloss',
        'eval_metric': 'PRAUC', # PR AUC를 직접 모니터링
        'bootstrap_type': 'Bernoulli',
        'verbose': 0,
        'random_seed': Config.RANDOM_STATE,
        'iterations': trial.suggest_int('iterations', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'depth': trial.suggest_int('depth', 3, 12),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True), # (lambda_l2)
        #'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 1.0), # (feature_fraction)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # (bagging_fraction)
    }

    # 🌟 변경점: CatBoostClassifier 사용 및 GPU 설정 (task_type='GPU')
    model = cb.CatBoostClassifier(task_type='GPU', **params)

    # 🌟 변경점: CatBoost의 Early Stopping 방식으로 수정
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              early_stopping_rounds=100,
              verbose=False)

    # CatBoost는 PRAUC를 직접 계산하므로, 최적화 단계의 점수를 바로 반환
    score = model.get_best_score()['validation']['PRAUC']

    return score


# === 4. 메인 실행 블록 ===
if __name__ == '__main__':

    # --- Step 2: 베이스라인 모델 성능 측정 ---
    print("\n" + "="*50)
    print("📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)")
    print("="*50)

    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    baseline_model = cb.CatBoostClassifier(task_type='GPU', random_seed=config.RANDOM_STATE, verbose=0)
    baseline_model.fit(X_train, y_train)

    y_pred_proba_base = baseline_model.predict_proba(X_test)[:, 1]
    y_pred_class_base = (y_pred_proba_base > 0.5).astype(int)

    final_results = {}
    final_results['Baseline'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_base),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_base),
        "F1-Score": f1_score(y_test, y_pred_class_base),
    }
    print("✅ 베이스라인 모델 평가 완료.")

    # --- Step 3: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...")
    print("="*50)

    study = optuna.create_study(direction='maximize')
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")
    study.optimize(lambda trial: objective(trial, X_train, y_train),
                   n_trials=config.N_TRIALS,
                   callbacks=[lambda study, trial: pbar.update(1)])
    pbar.close()

    # --- Step 4: 최적 하이퍼파라미터 명시적 출력 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터 (Best Hyperparameters)")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    # --- Step 5: 최종 모델 학습 및 평가 ---
    print(f"\n🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...")
    # 🌟 변경점: lgb.LGBMClassifier -> cb.CatBoostClassifier, task_type='GPU'
    final_model = cb.CatBoostClassifier(task_type='GPU', objective='Logloss', verbose=0, random_seed=config.RANDOM_STATE,bootstrap_type='Bernoulli', **best_params)
    final_model.fit(X_train, y_train)

    y_pred_proba_tuned = final_model.predict_proba(X_test)[:, 1]
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    final_results['Optuna Tuned'] = {
        "PR AUC": average_precision_score(y_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 모델 평가 완료.")

    # --- 최종 성능 비교 ---
    print("\n" + "="*60)
    print("📊 최종 성능 비교 결과 (Test Set)")
    print("="*60)
    results_df = pd.DataFrame(final_results).T
    print(results_df.round(4))

    # 🌟 --- Step 6: 전체 데이터에 대한 예측 결과 원본 CSV에 추가 후 저장 ---
    print("\n" + "="*60)
    print("💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    # 1. 학습 데이터(Train Set)에 대한 예측 수행
    train_pred_proba = final_model.predict_proba(X_train)[:, 1]
    train_pred_class = (train_pred_proba > 0.5).astype(int)

    # 2. 원본 DataFrame에 새로운 컬럼 추가 (초기값은 비워둠)
    df['s2_pred_proba'] = np.nan

    # 3. 분할 시 사용했던 인덱스를 이용해 예측 결과를 원래 위치에 채워넣기
    df.loc[train_indices, 's2_pred_proba'] = train_pred_proba
    df.loc[test_indices, 's2_pred_proba'] = y_pred_proba_tuned

    # 4. 새로운 CSV 파일로 저장
    output_filename = os.path.join('/content/drive/MyDrive/PADA model code/PADA/results/s2/hotel/CatBoost', "BERT_hotel_with_s2_predictions_CatBoost.csv")
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"✅ 모든 데이터의 예측 결과가 '{output_filename}' 파일에 성공적으로 저장되었습니다.")
    print("\n🎉 모든 과정이 완료되었습니다!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Step 1: 데이터 로드 및 분할 중...
✅ 완료 (학습용: 71604건, 테스트용: 17901건)

📊 Step 2: 베이스라인 모델 성능 측정 (GPU 사용)


[I 2025-10-12 17:33:53,401] A new study created in memory with name: no-name-50f45bdb-0693-48d5-a836-c3792958e035


✅ 베이스라인 모델 평가 완료.

🔬 Step 3: Optuna 하이퍼파라미터 튜닝 시작 (GPU 사용)...


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 17:33:58,163] Trial 0 finished with value: 0.14781774767366426 and parameters: {'iterations': 2000, 'learning_rate': 0.03810086482018409, 'depth': 7, 'l2_leaf_reg': 8.953011426975045e-08, 'subsample': 0.6798054836059031}. Best is trial 0 with value: 0.14781774767366426.
Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
[I 2025-10-12 17:34:01,203] Trial 1 finished with value: 0.23532266179294864 and parameters: {'iterations': 800, 'learning_rate': 0.23397366384846116, 'depth': 4, 'l2_leaf_reg': 4.938846627323946e-08, 'subsample': 0.6483047324025615}. Best is trial 1 with value: 0.23532266179294864.
Default metric period is 5 because PRAUC 


✅ 튜닝 완료!

🔬 최적 하이퍼파라미터 (Best Hyperparameters)
          iterations: 800
       learning_rate: 0.23397366384846116
               depth: 4
         l2_leaf_reg: 4.938846627323946e-08
           subsample: 0.6483047324025615

🔬 Step 5: 튜닝된 최종 모델 학습 및 평가...
✅ 튜닝된 모델 평가 완료.

📊 최종 성능 비교 결과 (Test Set)
              PR AUC  ROC AUC  F1-Score
Baseline      0.1635   0.6276    0.0036
Optuna Tuned  0.1343   0.5929    0.0331

💾 Step 6: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장
✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/PADA model code/PADA/results/s2/hotel/CatBoost/BERT_hotel_with_s2_predictions_CatBoost.csv' 파일에 성공적으로 저장되었습니다.

🎉 모든 과정이 완료되었습니다!
